In [26]:
import numpy as np
import cv2
import os
import pandas as pd

In [27]:
tabel = pd.read_csv('regions.csv')
kuda = tabel.loc[2]
kuda

cctv_folder_parent    SIMPANG 4 PATUNG KUDA
map_image_name        SIMPANG 4 PATUNG KUDA
Name: 2, dtype: object

In [22]:
kuda_cctv_path = 'cctv\\' + kuda.loc['cctv_folder_parent']
kuda_cctv_map  = 'map\\' + kuda.loc['cctv_folder_parent']

The perspective warp points is based on this equation:

$\begin{bmatrix}x'\\y'\\1\end{bmatrix} = 
\begin{bmatrix}m_{0,0} & m_{0,1} & m_{0,2}\\m_{1,0} & m_{1,1} & m_{1,2}\\m_{2,0} & m_{2,1} & m_{2,2}\end{bmatrix} *
\begin{bmatrix}x\\y\\1\end{bmatrix}$

In [126]:
def warpPerspectivePoints(pts, M):
    # NOTES: Please implement this in a way that does not involve
    # for loop :(
    #pts_warp = []
    #for p_i in pts:
    #    pts_x = M[0,0] * p_i[0] + M[0,1] * p_i[1] + M[0, 2]
    #    pts_y = M[1,0] * p_i[0] + M[1,1] * p_i[1] + M[1, 2]
    #    pts_div = M[2,0] * p_i[0] + M[2,1] * p_i[1] + M[2, 2]
    #    pts_new = np.float32([pts_x, pts_y])
    #    pts_new /= pts_div
    #    pts_warp.append(pts_new)
    #return np.array(pts_warp)
    
    trans = np.hstack((pts, np.ones((4,1)))) # Add ones at the bottom of matrix
    trans = np.matmul(M, np.transpose(trans))
    return np.transpose(trans[0:2,:] / trans[2:])

In [127]:
parent_folder = kuda_cctv_path
image_map = kuda_cctv_map
 
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(parent_folder + '\patungkuda selatan 1200_x264.mp4')
map = cv2.imread(image_map)
 
# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

x = 0
dim_x, dim_y = None, None

pts1 = np.float32([[97, 273],[363, 236], [1074, 476], [187, 770]])
pts2 = np.float32([[0, 0], [1080, 0], [1080, 720], [0, 720]])/2

M = cv2.getPerspectiveTransform(pts1,pts2)
pts_transform = np.int32(warpPerspectivePoints(pts1, M))

ii = 0
ord_q = ord('q') # For quit button
ord_p = ord('p') # For pause button

dim_x = np.max(pts2[:,0])
dim_y = np.max(pts2[:,1])

finished = False
try:
    while(cap.isOpened() and not finished):
        ret, frame = cap.read()
        if ret == True:

            f = frame
            dst = cv2.warpPerspective(f,M,(dim_x, dim_y))
            
            for i in pts_transform:
                cv2.circle(dst, tuple(i), 9, (0,255,0), -1)
            
            for i in pts1:
                cv2.circle(f, tuple(i), 9, (0,255,0), -1)
                
            cv2.imshow('Frame2',f)
            cv2.imshow('Frame', dst)
            
            # Press Q on keyboard to  exit
            k = cv2.waitKey(25) & 0xFF
            finished = k == ord_q       
            if k == ord_p:
                while True:
                    k = cv2.waitKey(25) & 0xFF
                    finished = k == ord_q        
                    
                    if k == ord_p or k == ord_q:
                        break
            ii += 1

        # Break the loop
        else: 
            break
except KeyboardInterrupt:
    pass
finally:
    # When everything done, release the video capture object
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()